# IA2A Peru Bank Credit Risk Evaluation Competition

This notebook preprocesses the dataset for the IA2A Peru Bank Credit Risk Evaluation competition, using SQL queries for initial data manipulation, followed by training a machine learning model using XGBoost. The model's performance is evaluated using the Mean FBeta-Score.

Steps:
1. Load and preprocess the data using SQL queries.
2. Split the data into training and validation sets.
3. Train an XGBoost model.
4. Evaluate the model's performance using the Mean FBeta-Score.
5. Generate predictions for the test dataset and prepare the submission file.

Author: Kevin Juan Román Rafaele

Date: 2024-07-23

License: MIT License

GitHub: https://github.com/PoppinElo/I2A2-Peru-comp

## Import libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import fbeta_score

## Load the data

**Load the data**

In [ ]:
train_path = "/content/I2A2-Peru/train.csv"
test_path = "/content/I2A2-Peru/test_without_labels.csv"
sample_submission_path = "/content/I2A2-Peru/sample_submission.csv"

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
sample_submission_data = pd.read_csv(sample_submission_path)

In [ ]:
train_data.head()

,id,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account_or_bounds,present_employment_since,installment_rate_in_percentage_of_disposable_income,personal_status_and_sex,...,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker,good_credit_risk
0,10,0 <= ... < 200 DM,12,existing credits paid back duly till now,car (new),1295,... < 100 DM,... < 1 year,3,female : divorced/separated/married,...,car or other,25,none,rent,1,skilled employee / official,1,none,yes,1
1,82,no checking account,18,existing credits paid back duly till now,business,1568,100 <= ... < 500 DM,1 <= ... < 4 years,3,female : divorced/separated/married,...,building society savings agreement / life insu...,24,none,rent,1,unskilled - resident,1,none,yes,0
2,827,no checking account,18,no credits taken/ all credits paid back duly,business,4165,... < 100 DM,1 <= ... < 4 years,2,male : single,...,car or other,36,stores,own,2,skilled employee / official,2,none,yes,1
3,410,0 <= ... < 200 DM,24,existing credits paid back duly till now,radio/television,1967,... < 100 DM,.. >= 7 years,4,female : divorced/separated/married,...,car or other,20,none,own,1,skilled employee / official,1,yes,yes,0
4,48,no checking account,11,critical account/ other credits existing (not ...,car (new),7228,... < 100 DM,1 <= ... < 4 years,1,male : single,...,building society savings agreement / life insu...,39,none,own,2,unskilled - resident,1,none,yes,0


In [ ]:
test_data.head()

,id,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account_or_bounds,present_employment_since,installment_rate_in_percentage_of_disposable_income,personal_status_and_sex,...,present_residence_since,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker
0,115,no checking account,48,critical account/ other credits existing (not ...,radio/television,3578,unknown/ no savings account,.. >= 7 years,4,male : single,...,1,real estate,47,none,own,1,skilled employee / official,1,yes,yes
1,346,0 <= ... < 200 DM,13,critical account/ other credits existing (not ...,radio/television,882,... < 100 DM,... < 1 year,4,male : single,...,4,real estate,23,none,own,2,skilled employee / official,1,none,yes
2,328,... >= 200 DM / salary assignments for at leas...,36,existing credits paid back duly till now,radio/television,4473,... < 100 DM,.. >= 7 years,4,male : single,...,2,car or other,31,none,own,1,skilled employee / official,1,none,yes
3,974,no checking account,30,critical account/ other credits existing (not ...,radio/television,2831,... < 100 DM,1 <= ... < 4 years,4,female : divorced/separated/married,...,2,car or other,33,none,own,1,skilled employee / official,1,yes,yes
4,587,... < 0 DM,12,existing credits paid back duly till now,furniture/equipment,1289,... < 100 DM,1 <= ... < 4 years,4,male : single,...,1,building society savings agreement / life insu...,21,none,own,1,unskilled - resident,1,none,yes


**Ensure test_data includes the id column for submission**

In [ ]:
test_data_ids = test_data['id']
test_data_ids.head()

0    115
1    346
2    328
3    974
4    587
Name: id, dtype: int64

## Querying with SQL

**Create an SQLite database in memory**

In [ ]:
conn = sqlite3.connect(':memory:')
c = conn.cursor()

**Load data into the database**

In [ ]:
train_data.to_sql('train_data', conn, index=False, if_exists='replace')
test_data.to_sql('test_data', conn, index=False, if_exists='replace')

ProgrammingError: Cannot operate on a closed database.

**Example SQL query: Get the average age in years grouped by good credit risk in the training set**

In [ ]:
query = """
SELECT good_credit_risk, AVG(age_in_years) AS avg_age_in_years
FROM train_data
GROUP BY good_credit_risk
"""
avg_age_in_years = pd.read_sql_query(query, conn)
print("\nAverage Age grouped by credit risk:")
print(avg_age_in_years)

ProgrammingError: Cannot operate on a closed database.

**Example SQL query: Get the types of housing in the training set**

In [ ]:
query = """
SELECT housing AS housing_type, count(housing) AS num_housing
FROM train_data
GROUP BY housing
"""
housing_types = pd.read_sql_query(query, conn)
print("\nHousing Types:")
print(housing_types)

ProgrammingError: Cannot operate on a closed database.

**Example SQL query: Get the number of people with good and bad credit risk grouped by sex and by personal status**

In [ ]:
query = """
SELECT
    CASE
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'female' THEN 'female'
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'male' THEN 'male'
        ELSE 'unknown'
    END AS sex,
    SUBSTR(personal_status_and_sex, INSTR(personal_status_and_sex, ': ') + 2) AS status,
    good_credit_risk
FROM
    train_data
    LIMIT 10;
"""
good_credit_risk = pd.read_sql_query(query, conn)
print("\nPeople with Good and Bad Credit Risk grouped by Sex and Personal Status:")
print(good_credit_risk)

ProgrammingError: Cannot operate on a closed database.

**Add the new columns in the train_data**

In [ ]:
c.execute("""
ALTER TABLE train_data
ADD COLUMN sex TEXT;
""")
c.execute("""
ALTER TABLE train_data
ADD COLUMN status TEXT;
""")

# Commit the changes
conn.commit()

**Update the table to populate the new columns in the train_data**

In [ ]:
c.execute("""
UPDATE train_data
SET
    sex = CASE
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'female' THEN 'female'
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'male' THEN 'male'
        ELSE 'unknown'
    END,
    status = SUBSTR(personal_status_and_sex, INSTR(personal_status_and_sex, ': ') + 2);
""")

# Commit the changes
conn.commit()

In [ ]:
query = """
SELECT id, sex, status, personal_status_and_sex
FROM train_data
LIMIT 5
"""
sex_and_status = pd.read_sql_query(query, conn)
print("\nSex and status separated:")
print(sex_and_status)


Sex and status separated:
    id     sex                      status  \
0   10  female  divorced/separated/married   
1   82  female  divorced/separated/married   
2  827    male                      single   
3  410  female  divorced/separated/married   
4   48    male                      single   

               personal_status_and_sex  
0  female : divorced/separated/married  
1  female : divorced/separated/married  
2                        male : single  
3  female : divorced/separated/married  
4                        male : single  


SQLite doesn't support dropping columns directly
To remove a column, you'll need to create a new table without the column, copy the data, and then rename the new table to the old table's name.


In [ ]:
# Create a new table without the `personal_status_and_sex` column
c.execute("""
CREATE TABLE new_train_table AS
SELECT id, status_of_existing_checking_account, duration_in_month,
       credit_history, purpose, credit_amount,
       savings_account_or_bounds, present_employment_since,
       installment_rate_in_percentage_of_disposable_income,
       sex, status, other_debtors_or_guarantors,
       present_residence_since, property, age_in_years,
       other_installment_plans, housing,
       number_of_existing_credits_at_this_bank, job,
       number_of_people_being_liable_to_provide_maintenance_for,
       telephone, foreign_worker, good_credit_risk
FROM train_data;
""")

# Drop the old table
c.execute("DROP TABLE train_data;")

# Rename the new table to the old table's name
c.execute("ALTER TABLE new_train_table RENAME TO train_data;")

# Commit the changes
conn.commit()

ProgrammingError: Cannot operate on a closed database.

In [ ]:
new_train_data = pd.read_sql_query("SELECT * FROM train_data", conn)
new_train_data.head()

,id,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account_or_bounds,present_employment_since,installment_rate_in_percentage_of_disposable_income,sex,...,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker,good_credit_risk
0,10,0 <= ... < 200 DM,12,existing credits paid back duly till now,car (new),1295,... < 100 DM,... < 1 year,3,female,...,car or other,25,none,rent,1,skilled employee / official,1,none,yes,1
1,82,no checking account,18,existing credits paid back duly till now,business,1568,100 <= ... < 500 DM,1 <= ... < 4 years,3,female,...,building society savings agreement / life insu...,24,none,rent,1,unskilled - resident,1,none,yes,0
2,827,no checking account,18,no credits taken/ all credits paid back duly,business,4165,... < 100 DM,1 <= ... < 4 years,2,male,...,car or other,36,stores,own,2,skilled employee / official,2,none,yes,1
3,410,0 <= ... < 200 DM,24,existing credits paid back duly till now,radio/television,1967,... < 100 DM,.. >= 7 years,4,female,...,car or other,20,none,own,1,skilled employee / official,1,yes,yes,0
4,48,no checking account,11,critical account/ other credits existing (not ...,car (new),7228,... < 100 DM,1 <= ... < 4 years,1,male,...,building society savings agreement / life insu...,39,none,own,2,unskilled - resident,1,none,yes,0


**Add the new columns in the test_data**

In [ ]:
c.execute("""
ALTER TABLE test_data
ADD COLUMN sex TEXT;
""")
c.execute("""
ALTER TABLE test_data
ADD COLUMN status TEXT;
""")

# Commit the changes
conn.commit()

ProgrammingError: Cannot operate on a closed database.

**Update the table to populate the new columns in the test_data**

In [ ]:
c.execute("""
UPDATE test_data
SET
    sex = CASE
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'female' THEN 'female'
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'male' THEN 'male'
        ELSE 'unknown'
    END,
    status = SUBSTR(personal_status_and_sex, INSTR(personal_status_and_sex, ': ') + 2);
""")

# Commit the changes
conn.commit()

ProgrammingError: Cannot operate on a closed database.

In [ ]:
query = """
SELECT id, sex, status, personal_status_and_sex
FROM test_data
LIMIT 5
"""
sex_and_status = pd.read_sql_query(query, conn)
print("\nSex and status separated:")
print(sex_and_status)


Sex and status separated:
    id     sex                      status  \
0  115    male                      single   
1  346    male                      single   
2  328    male                      single   
3  974  female  divorced/separated/married   
4  587    male                      single   

               personal_status_and_sex  
0                        male : single  
1                        male : single  
2                        male : single  
3  female : divorced/separated/married  
4                        male : single  


SQLite doesn't support dropping columns directly.
To remove a column, you'll need to create a new table without the column, copy the data, and then rename the new table to the old table's name.

In [ ]:
# Create a new table without the `personal_status_and_sex` column
c.execute("""
CREATE TABLE new_test_table AS
SELECT id, status_of_existing_checking_account, duration_in_month,
       credit_history, purpose, credit_amount,
       savings_account_or_bounds, present_employment_since,
       installment_rate_in_percentage_of_disposable_income,
       sex, status, other_debtors_or_guarantors,
       present_residence_since, property, age_in_years,
       other_installment_plans, housing,
       number_of_existing_credits_at_this_bank, job,
       number_of_people_being_liable_to_provide_maintenance_for,
       telephone, foreign_worker
FROM test_data;
""")

# Drop the old table
c.execute("DROP TABLE test_data;")

# Rename the new table to the old table's name
c.execute("ALTER TABLE new_test_table RENAME TO test_data;")

# Commit the changes
conn.commit()

In [ ]:
new_test_data = pd.read_sql_query("SELECT * FROM test_data", conn)
new_test_data.head()

,id,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account_or_bounds,present_employment_since,installment_rate_in_percentage_of_disposable_income,sex,...,present_residence_since,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker
0,115,no checking account,48,critical account/ other credits existing (not ...,radio/television,3578,unknown/ no savings account,.. >= 7 years,4,male,...,1,real estate,47,none,own,1,skilled employee / official,1,yes,yes
1,346,0 <= ... < 200 DM,13,critical account/ other credits existing (not ...,radio/television,882,... < 100 DM,... < 1 year,4,male,...,4,real estate,23,none,own,2,skilled employee / official,1,none,yes
2,328,... >= 200 DM / salary assignments for at leas...,36,existing credits paid back duly till now,radio/television,4473,... < 100 DM,.. >= 7 years,4,male,...,2,car or other,31,none,own,1,skilled employee / official,1,none,yes
3,974,no checking account,30,critical account/ other credits existing (not ...,radio/television,2831,... < 100 DM,1 <= ... < 4 years,4,female,...,2,car or other,33,none,own,1,skilled employee / official,1,yes,yes
4,587,... < 0 DM,12,existing credits paid back duly till now,furniture/equipment,1289,... < 100 DM,1 <= ... < 4 years,4,male,...,1,building society savings agreement / life insu...,21,none,own,1,unskilled - resident,1,none,yes


**Close the SQLite connection**

In [ ]:
conn.close()

## Data Preprocessing

**Define the data preprocessing function**

In [ ]:
def preprocess_data(data):
    # Handle missing values
    data = data.fillna(-999)  # Simplest way to handle missing values
    # Encode categorical variables
    categorical_features = data.select_dtypes(include=['object']).columns
    for col in categorical_features:
        data[col] = data[col].astype('category').cat.codes
    return data

**Preprocess the data**

In [ ]:
train_data = preprocess_data(new_train_data)
test_data = preprocess_data(new_test_data)

**Separate features and target variable**

In [ ]:
X = train_data.drop('good_credit_risk', axis=1)
y = train_data['good_credit_risk']

**Split the data into training and validation sets**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

**Feature Scaling**

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
test_data = scaler.transform(test_data)

## Modeling

**Define the Mean FBeta-Score function**

In [ ]:
def mean_fbeta_score(y_true, y_pred, beta=1.0):
    return fbeta_score(y_true, y_pred, beta=beta, average='macro')

**Model Building**

Try different classifiers

In [ ]:
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

**Train and evaluate models**

In [ ]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    fbeta = mean_fbeta_score(y_val, y_pred, beta=1.0)  # You can change beta as needed
    print(f'{name} - FBeta-Score: {fbeta:.4f}')

RandomForest - FBeta-Score: 0.5494
XGBoost - FBeta-Score: 0.6387


**Choose the best model and predict on the test set**

In [ ]:
best_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
best_model.fit(X_train, y_train)
test_predictions = best_model.predict(test_data)

**Prepare the submission file**

In [ ]:
submission = pd.DataFrame({'id': test_data_ids, 'good_credit_risk': test_predictions})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

,id,good_credit_risk
0,115,0
1,346,0
2,328,1
3,974,0
4,587,0
